In [73]:
import numpy as np
import sympy as sp
from scipy.linalg import eig
from scipy.integrate import quad, dblquad

from math import sin, cos, pi

In [74]:
#Properties (N and mm)
E1 = 60800
E2 = 58250
v12 = 0.07
G12 = 4550
t = 0.21
v21 = v12 * E2 / E1

# Plate Parameters
a = 360
b = 360
m = 3
n = 3

#a, b = sp.symbols(['a', 'b'])

# Loads
Nxx = -1
Nyy = 0
Nxy = 0

In [75]:
#Boundary conditions
u1tx = 0 ; u1rx = 1 ; u2tx = 0 ; u2rx = 1
v1tx = 0 ; v1rx = 1 ; v2tx = 0 ; v2rx = 1
w1tx = 0 ; w1rx = 1 ; w2tx = 0 ; w2rx = 1
u1ty = 0 ; u1ry = 1 ; u2ty = 0 ; u2ry = 1
v1ty = 0 ; v1ry = 1 ; v2ty = 0 ; v2ry = 1
w1ty = 0 ; w1ry = 1 ; w2ty = 0 ; w2ry = 1


**[Q] in the principal direction**  pg. 52 chap 2 of the three regions

In [76]:
Q_0 = np.array([[E1/(1-v12*v21), v12*E2/(1-v12*v21), 0],
                 [v12*E2/(1-v12*v21), E2/(1-v12*v21),0],
                 [0, 0, G12]])

**[Q] rotated in theta direction** pg. 53 and 54 of chap 2

In [77]:
layup = [(45,'f'),(0,'f'),(0,'f'),(45,'f'),(0,'f'),(0,'f'),(45,'f')]
Q_layup = []

#Layup assembly
for theta in layup:
    c = np.cos(theta[0]*np.pi/180)
    s = np.sin(theta[0]*np.pi/180)

    T_real = np.array([[c**2, s**2, 2*c*s],
                       [s**2, c**2, -2*c*s],
                       [-c*s, c*s, c**2-s**2]])

    T_engineering =  np.array([[c**2, s**2, c*s],
                               [s**2, c**2, -c*s],
                               [-2*c*s, 2*c*s, c**2-s**2]])


    if theta[1] == 'f':
        Q_layup.append((np.linalg.inv(T_real))@Q_0@T_engineering)


**[A], [B], [D]** pg. 24 fo chap 4 of the three regions

In [78]:
t_position = [-3.5*t, -2.5*t, -1.5*t, -0.5*t, 0.5*t, 1.5*t, 2.5*t, 3.5*t]

A = np.zeros(9).reshape(3,3)
B = np.zeros(9).reshape(3,3)
D = np.zeros(9).reshape(3,3)

for i in enumerate(Q_layup):
    zk1 = t_position[i[0]+1]
    zk0 = t_position[i[0]]
    
    A += (zk1 - zk0)*i[1]
    B += (1/2)*(zk1**2 - zk0**2)*i[1]
    D += (1/3)*(zk1**3 - zk0**3)*i[1]


F = np.vstack([
    np.hstack([A, B]),
    np.hstack([B, D])
])

F = sp.Matrix(F)

### Rayleigh-Ritz formulation

In [79]:
Ny = sp.symbols('Ny')
xi, eta  = sp.symbols(['xi', 'eta'])

In [80]:
xi**4/8 - xi**2/4 + 0.125

xi**4/8 - xi**2/4 + 0.125

In [81]:
# Bardell terms
# Su
uf_xi = {0: u1tx*(0.25*xi**3 - 0.75*xi + 0.5),
        1: u1rx*(0.125*xi**3 - 0.125*xi**2 - 0.125*xi + 0.125),
        2: u2tx*(-0.25*xi**3 + 0.75*xi + 0.5),
        3: u2rx*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3)}

uf_eta = {0: u1ty*(0.25*eta**3 - 0.75*eta + 0.5),
        1: u1ry*(0.125*eta**3 - 0.125*eta**2 - 0.125*eta + 0.125),
        2: u2ty*(-0.25*eta**3 + 0.75*eta + 0.5),
        3: u2ry*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3)}

# Sv
vf_xi = {0: v1tx*(0.25*xi**3 - 0.75*xi + 0.5),
        1: v1rx*(0.125*xi**3 - 0.125*xi**2 - 0.125*xi + 0.125),
        2: v2tx*(-0.25*xi**3 + 0.75*xi + 0.5),
        3: v2rx*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3)}

vf_eta = {0: v1ty*(0.25*eta**3 - 0.75*eta + 0.5),
        1: v1ry*(0.125*eta**3 - 0.125*eta**2 - 0.125*eta + 0.125),
        2: v2ty*(-0.25*eta**3 + 0.75*eta + 0.5),
        3: v2ry*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3)}

#Sw
wf_xi = {0: w1tx*(0.25*xi**3 - 0.75*xi + 0.5),
        1: w1rx*(0.125*xi**3 - 0.125*xi**2 - 0.125*xi + 0.125),
        2: w2tx*(-0.25*xi**3 + 0.75*xi + 0.5),
        3: w2rx*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3)}

wf_eta = {0: w1ty*(0.25*eta**3 - 0.75*eta + 0.5),
        1: w1ry*(0.125*eta**3 - 0.125*eta**2 - 0.125*eta + 0.125),
        2: w2ty*(-0.25*eta**3 + 0.75*eta + 0.5),
        3: w2ry*(-1/8 - 1/8*xi + 1/8*xi**2 + 1/8*xi**3)}


for i in range(10):
        k = i # + 4
        fxi_k = sp.sin((i+1)*sp.pi*(xi+1)/2)
        feta_k = sp.sin((i+1)*sp.pi*(eta+1)/2)

        uf_xi[k] = fxi_k
        uf_eta[k] = feta_k
        vf_xi[k] = fxi_k
        vf_eta[k] = feta_k
        wf_xi[k] = fxi_k
        wf_eta[k] = feta_k

In [82]:
Su, Sv, Sw = [[]], [[]], [[]]

for i in range(m):
    for j in range(n):
        Su[0].append(uf_xi[i]*uf_eta[j])
        Sv[0].append(vf_xi[i]*vf_eta[j])
        Sw[0].append(wf_xi[i]*wf_eta[j])
Su, Sv, Sw = sp.Matrix(Su), sp.Matrix(Sv), sp.Matrix(Sw)


B0_11 = np.array((2/a) * sp.diff(Su, xi))
B0_22 = np.array((2/b) * sp.diff(Sv, eta))
B0_31 = np.array((2/b)*sp.diff(Su, eta))
B0_32 = np.array((2/a)*sp.diff(Sv, xi))
B0_43 = np.array(-(4/a**2) * sp.diff(Sw, xi, xi))
B0_53 = np.array(-(4/b**2) * sp.diff(Sw, eta, eta))
B0_63 = np.array(-2*(4/(a*b)) * sp.diff(Sw, xi, eta))
Z = np.zeros(m*n).reshape(1, m*n)

B0_kappa = sp.Matrix([
    np.hstack([*B0_11, *Z, *Z]),
    np.hstack([*Z, *B0_22, *Z]),
    np.hstack([*B0_31, *B0_32, *Z]),
    np.hstack([*Z, *Z, *B0_43]),
    np.hstack([*Z, *Z, *B0_53]),
    np.hstack([*Z, *Z, *B0_63])
])


F = sp.Matrix(F)


G = sp.Matrix([
    np.hstack([*Z, *Z, *sp.diff(Sw, xi)*(2/a)]),
    np.hstack([*Z, *Z, *sp.diff(Sw, eta)*(2/b)])
 ])


     

G = sp.Matrix(G)

N = sp.Matrix([[Nxx, Nxy],
                [Nxy, Nyy]])

In [83]:
Sw

Matrix([[sin(pi*(eta/2 + 1/2))*sin(pi*(xi/2 + 1/2)), sin(pi*(eta + 1))*sin(pi*(xi/2 + 1/2)), sin(pi*(3*eta/2 + 3/2))*sin(pi*(xi/2 + 1/2)), sin(pi*(eta/2 + 1/2))*sin(pi*(xi + 1)), sin(pi*(eta + 1))*sin(pi*(xi + 1)), sin(pi*(3*eta/2 + 3/2))*sin(pi*(xi + 1)), sin(pi*(eta/2 + 1/2))*sin(pi*(3*xi/2 + 3/2)), sin(pi*(eta + 1))*sin(pi*(3*xi/2 + 3/2)), sin(pi*(3*eta/2 + 3/2))*sin(pi*(3*xi/2 + 3/2))]])

In [84]:
G

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.00277777777777778*pi*sin(pi*(eta/2 + 1/2))*cos(pi*(xi/2 + 1/2)), 0.00277777777777778*pi*sin(pi*(eta + 1))*cos(pi*(xi/2 + 1/2)), 0.00277777777777778*pi*sin(pi*(3*eta/2 + 3/2))*cos(pi*(xi/2 + 1/2)), 0.00555555555555556*pi*sin(pi*(eta/2 + 1/2))*cos(pi*(xi + 1)), 0.00555555555555556*pi*sin(pi*(eta + 1))*cos(pi*(xi + 1)), 0.00555555555555556*pi*sin(pi*(3*eta/2 + 3/2))*cos(pi*(xi + 1)), 0.00833333333333333*pi*sin(pi*(eta/2 + 1/2))*cos(pi*(3*xi/2 + 3/2)), 0.00833333333333333*pi*sin(pi*(eta + 1))*cos(pi*(3*xi/2 + 3/2)), 0.00833333333333333*pi*sin(pi*(3*eta/2 + 3/2))*cos(pi*(3*xi/2 + 3/2))],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.00277777777777778*pi*sin(pi*(xi/2 + 1/2))*cos(pi*(eta/2 + 1/2)), 0.00555555555555556*pi*sin(pi*(xi/2 + 1/2))*cos(pi*(eta + 1)), 0.00833333333333333*pi*sin(pi*(xi/2 + 1/2))*cos(pi*(3*eta/2 + 3/2)), 0.00277777777777778*pi*sin(pi*(xi + 1))*cos(pi*(eta/2 + 1/2)), 0.00555555555555556*pi*sin(

In [85]:
# KG = sp.integrate(
#      sp.integrate(
#         G.T * N * G, (xi, -1, 1)), (eta, -1, 1)
# ) * (a*b/4)

# KG = np.array(KG, dtype=np.longdouble)

KG_int = (G.T * N * G) * (a*b/4)
KG = np.zeros((3*m*n)**2).reshape(3*m*n, 3*m*n)

for i in range(3*m*n):
    for j in range(3*m*n):
        KG[i,j] = dblquad(
            lambda xi, eta: eval(str(KG_int[i,j])), -1, 1, lambda xi: -1, lambda xi: 1)[0]

KG


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+

In [86]:
sp.Matrix(KG)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                     0,                     0,                     0,                     0,                     0,                     0,                     0,                     0,                     0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                     0,                     0,                     0,                     0,                     0,                     0,                     0,                     0,                     0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                     0,                     0,                     0,                     0,                     0,                     0,                     0,                     0,                     0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                     0,                     0,                     0,                     0,                     0,                     0,          

In [87]:
#KG[18, 18].subs(a, 360).subs(b, 360).subs(xi, 1).subs(eta, 1)

In [88]:
print(KG.shape)
print(KG.min())
print(KG.max())

(27, 27)
-22.206609902451056
6.661338147750939e-16


In [89]:
# K = sp.integrate(
#     sp.integrate(
#         B0_kappa.T * F * B0_kappa, (xi, -1, 1)), (eta, -1, 1)
# ) * (a*b/4)

# K = np.array(K, dtype=np.longdouble)


K_int = (B0_kappa.T * F * B0_kappa) * (a*b/4)
K = np.zeros((3*m*n)**2).reshape(3*m*n, 3*m*n)

for i in range(3*m*n):
    for j in range(3*m*n):
        K[i,j] = dblquad(
            lambda xi, eta: eval(str(K_int[i,j])), -1, 1, lambda xi: -1, lambda xi: 1)[0]

K

C:\Users\Pichau\AppData\Roaming\Python\Python39\site-packages\scipy\integrate\_quadpack_py.py:1151: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


array([[ 2.36078507e+05,  8.28121643e-12, -1.52340363e-11,
         7.30388999e-12, -1.43473535e+03, -1.25710630e-12,
        -2.31687879e-11, -3.63734364e-13,  4.84728790e-13,
         1.99128802e+03, -5.40273813e-13, -4.26325641e-13,
        -8.37303685e-14, -7.47989017e+04,  6.72983168e-12,
         7.45906500e-13,  3.92818667e-12, -5.01989626e-12,
        -4.32921809e-32, -5.40214928e-32,  2.76942939e-31,
         7.48029409e-14,  1.51051173e-30,  7.88860905e-31,
        -1.01466532e-29,  1.89573561e-31, -9.16486222e-31],
       [ 2.95829658e-12,  3.94266131e+05,  5.86856265e-11,
         1.43473535e+03,  1.02285654e-11, -2.58252363e+03,
        -1.88247026e-12, -1.88342924e-11, -1.07973736e-12,
         7.88385943e-13,  4.97822004e+03, -1.25202692e-12,
         7.47989017e+04,  3.89238429e-13, -1.34638023e+05,
        -3.67869919e-12,  3.87059454e-13,  1.09003260e-11,
        -6.21813248e-32, -4.33306733e-31,  3.95372766e-32,
         1.55864943e-31,  9.97372546e-14,  1.55467722e-

In [90]:
print(K.shape)
print(K.min())
print(K.max())

(27, 27)
-242348.4413950852
2124706.56406355


In [91]:
A = K
B = KG

eig_values, eig_vectors = eig(A, B)

eig_values

array([         inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
       -94.5038418 +0.j, -42.79986218+0.j, -37.28932529+0.j,
        -6.21534287+0.j, -24.58150038+0.j,  -4.14282527+0.j,
       -10.66564523+0.j, -19.15472452+0.j, -16.5715993 +0.j])

In [92]:
eig_values.min()

(-94.50384180363099+0j)

In [93]:
eig_values

array([         inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
                inf+0.j,          inf+0.j,          inf+0.j,
       -94.5038418 +0.j, -42.79986218+0.j, -37.28932529+0.j,
        -6.21534287+0.j, -24.58150038+0.j,  -4.14282527+0.j,
       -10.66564523+0.j, -19.15472452+0.j, -16.5715993 +0.j])

In [94]:
-16.57243666/4

-4.143109165

In [95]:
66.28974663/4

16.5724366575

In [96]:
eigvals, eigvecs = lb(A, B, silent=True)
eigvals

NameError: name 'lb' is not defined

## Check with compmech

In [ ]:
from compmech.panel.panel import Panel
from compmech.analysis import lb, static

In [ ]:
# skin panels
laminaprop = (E1, E2, v12, G12, G12, G12)
stack = [45, 0, 0, 45, 0, 0, 45,]
p1 = Panel(group='skin', Nxx=-1, x0=0, y0=0., a=a, b=b,m=7, n=7, plyt=t, stack=stack, laminaprop=laminaprop)

In [ ]:
# boundary conditions CCCC
p1.u1tx = 0 ; p1.u1rx = 1 ; p1.u2tx = 0 ; p1.u2rx = 1
p1.v1tx = 0 ; p1.v1rx = 1 ; p1.v2tx = 0 ; p1.v2rx = 1
p1.w1tx = 0 ; p1.w1rx = 1 ; p1.w2tx = 0 ; p1.w2rx = 1
p1.u1ty = 0 ; p1.u1ry = 1 ; p1.u2ty = 0 ; p1.u2ry = 1
p1.v1ty = 0 ; p1.v1ry = 1 ; p1.v2ty = 0 ; p1.v2ry = 1
p1.w1ty = 0 ; p1.w1ry = 1 ; p1.w2ty = 0 ; p1.w2ry = 1

In [ ]:
k0_1 = p1.calc_k0()
kG0_1 = p1.calc_kG0()

eigvals, eigvecs = lb(k0_1, kG0_1, silent=True)
eigvals

In [ ]:
print(k0_1.min())
print(k0_1.max())

In [ ]:
print(kG0_1.min())
print(kG0_1.max())

In [ ]:
sp.Matrix(kG0_1.toarray())

In [ ]:
sp.Matrix(KG)

## Check with composipy

In [ ]:
from composipy import Laminate, Ply, buckling_load

In [ ]:
ply = Ply(E1, E2, v12, G12, t)
layup_1 = [(45, ply),(0, ply),(0, ply),(45, ply),(0, ply),(0, ply),(45, ply)]
laminate_1 = Laminate(layup_1)

In [ ]:
load = buckling_load(a, b, laminate_1.D, shape_plot = False , eig = True, n=10)

In [ ]:
a = abs(load['eigen values'])
a.sort()
a